## Scrapping urbania

The goal of this notebook is to scrape data from https://urbania.pe/ and store it on a mongo databse

### Libraries import

In [1]:
import math
import numpy as np
import pymongo
import re 
import urllib.request as urllib2
import datetime

from time import sleep
from bs4 import BeautifulSoup

from constants import *

In [2]:
# bsae urls to use
base_url = 'https://urbania.pe{}'
apartments_url = 'https://urbania.pe/buscar/venta-de-departamentos'


### Define database connections

In [5]:
uri = "mongodb://{}:{}@{}/{}".format(username, password, hostlist, database)
client = pymongo.MongoClient(uri)
db = client['urbania']

In [29]:
list(db['admin'].find())

[{'_id': ObjectId('5deb3aba3dadd940d2148a36'), 'name': 'gonzalo'},
 {'_id': ObjectId('5dec891299df02824e747081'), 'name': 'jay'}]

In [7]:
apartmentsdb = db['apartments']

### Here we put the logic to get the data

In [13]:
def getSoupObj(url):
    sleep(1.5)
    try:
        r = urllib2.urlopen(url)
    except UnicodeEncodeError as e:
        return None
    html = r.read()
    htmltext = html.decode('ISO-8859-2')
    return BeautifulSoup(htmltext, 'lxml')

In [159]:
dictionary = {
    'Tipos Dptos.': 'apartment_type',
    'Dpto.': 'department',
    'Provincia': 'province',
    'Distrito': 'district',
    'Urb.': 'urbanization',
    'Torres': 'towers',
    'Inmuebles': 'properties',
    'Ascensores': 'elevators',
    'NÂ°pisos': 'n_floors',
    'Cocheras': 'garages',
    'Referencia': 'reference',
    'Construye': 'builds',
    'Comercializa': 'sells',
    'Financia': 'finances'
}

In [199]:
tihs = 10000
count = 1
apartments = []
fails = 0
# count is for page counter
while count < 5:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    print(apartments_url)
    req = urllib2.Request(apartments_url, headers=hdr)
    apartmentsBsObj = getSoupObj(req)
    print(type(apartmentsBsObj))
#     objs = apartmentsBsObj.findAll('article', {'class': 'b-card-item js-card-item b-card-item--proyecto-destacado'})
     
    objs = apartmentsBsObj.findAll('div', {'class': 'posting-card highlighted'})
#     objsSeen = apartmentsBsObj
    print('In page {} we found {} apartments'.format(count, len(objs)))
    for obj in objs:
#         the objA is the object with <a> label in obj

        apartment['url'] = obj['data-to-posting']
        try:
            get_apartment(apartment)
        except:
            fails+=1
            print('damn so')
        print(apartment)
        apartments.append(apartment)
        
#         break

        
    count +=1
    break

https://urbania.pe/buscar/venta-de-departamentos
<class 'bs4.BeautifulSoup'>
In page 1 we found 15 apartments
{'url': '/inmueble/proyecto-vizuale-lima-lince-inmobiliaria-ortiz-empresa-8004416', 'title': 'Vizuale', 'address': 'AV. AREQUIPA 2426, Lince, Lima'}
{'url': '/inmueble/proyecto-jose-gonzales-685-nuevo-proyecto-miraflores-armendariz-constructora-titan-s-a-empresa-8004749', 'title': 'JosĂŠ Gonzales 685 - Nuevo Proyecto', 'address': 'Calle JosĂŠ Gonzales NÂ° 685, ARMENDARIZ, Miraflores'}
{'url': '/inmueble/proyecto-suburbia-toma-jesus-maria-lima-jesus-maria-al-punto-empresa-8004784', 'title': 'Suburbia, Toma JesĂşs Maria', 'address': 'Natalio Sanchez 287, JesĂşs MarĂ\xada, Lima'}
{'url': '/inmueble/proyecto-vanguard-en-construccion-lima-san-miguel-grupo-inmobiliario-brocksa-sac-empresa-8004161', 'title': 'Vanguard - ÂĄEn ConstrucciĂłn', 'address': 'JirĂłn Sucre 597, San Miguel, Lima'}
{'url': '/inmueble/proyecto-inclan-1010-ultimo-flat-con-entrega-inmediata-en-miraflores-lima-mira

In [196]:
fails

5

In [ ]:
class building:
    def __init__(self, buildingType):
        self.buildingType = buildingType
    
    

In [27]:
class apartment:
    def __init__(self):
        

SyntaxError: invalid syntax (<ipython-input-27-b7843a038499>, line 2)

In [198]:
def get_apartment(apartment):
    
    apartment_url = base_url.format(apartment['url'])
#     print(apartment_url)
    req = urllib2.Request(apartment_url)
    apartmentObj = getSoupObj(req)
#     print(apartmentObj)
    
    if apartmentObj == -1:
        return
#     print(apartment['url'])
    get_text(apartment, apartmentObj, 'title', 'info-title', 'h2')
    get_text(apartment, apartmentObj, 'address', 'info-location', 'h2')
#     get_text(apartment, apartmentObj, 'features', 'e-features', 'p')
#     get_text(apartment, apartmentObj, 'intial_price', 'e-totalPrice', 'p')
#     data_aux_general_info = apartmentObj.find('div', {'class': 'b-wrap-interbank u-flex-wrap'}).text
#     data_aux_real_state_info = apartmentObj.find('div', {'class': 'b-contributing-company u-flex-wrap'}).text
#     contributing_company_data = data_aux_real_state_info.split('\n\n')
#     general_info = data_aux_general_info.split('\n\n')
    
#     get_general_info(general_info, apartment)
#     get_general_info(contributing_company_data, apartment, 0)
#     print('send memes')
    
#     print(apartment)
#     print(address.text)


In [197]:
def get_text(obj, objBs, feature, tag, marker, inplace=True):
    new_feature = objBs.find(marker, {'class': tag})
#     print(new_feature)
    if inplace:
        obj[feature] = new_feature.text
    else:
        return new_feature.text

In [179]:
def get_general_info(general_info, obj, option= 1):
    general_info = general_info[1:]
    general_info = general_info[:-1]
    
    if option:
        state_delivery = general_info[0].split('\n')
        obj['status'] = state_delivery[0]

        if len(state_delivery)>1:
            obj['delivery_date'] = state_delivery[1]
    index = option

    for i in range(index, len(general_info)):
    
        data_aux = general_info[i].split('\n')
        data_aux = remove_blanks(data_aux, '')
#         index_data = len(data_aux)
        obj[dictionary[data_aux[0]]]= data_aux[1]
        

In [178]:
def remove_blanks(current_list, pattern):
    final_list = [item for item in current_list if elem != pattern]
    return final_list

In [ ]:
def get_real_estate(real_estate):
    